In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

def compute_reward(self, robot_position, target_position, collision_status):
    reward = 0
    current_distance = np.linalg.norm(target_position - robot_position)

    # Reward for moving closer to the target
    if self.previous_distance > current_distance:
        reward += (self.previous_distance - current_distance) * 10
    else:
        reward -= 1  # Penalty for moving away from the target

    # Check for collision and apply a large penalty if a collision occurs
    if collision_status:
        reward -= 100  

    # Reward for reaching the target within a predefined threshold
    if current_distance < self.target_threshold:
        reward += 100  

    self.previous_distance = current_distance  # Update previous distance for next reward calculation
    return reward

class QNetwork(nn.Module):
    def __init__(self, input_size, output_size=2, hidden_size=128):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.relu = nn.ReLU()                          # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Hidden to output layer

    def forward(self, x):
        x = self.fc1(x)  # Input passes through the first fully connected layer
        x = self.relu(x)  # Activation function is applied
        x = self.fc2(x)   # Result passes through the output layer
        return x

# Assuming the use of this network in a training loop
def train(env, model, episodes, learning_rate=0.001, gamma=0.99):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            state_tensor = torch.FloatTensor(state)
            velocities = model(state_tensor)

            # Select action based on the model output
            linear_velocity = velocities[0].item()
            angular_velocity = velocities[1].item()
            action = (linear_velocity, angular_velocity)

            next_state, reward, done, _ = env.step(action)
            state = next_state

    return model

# Example to instantiate and use the model
input_size = 360  # Example for 360-degree LIDAR data points
hidden_size = 128  # Hidden layer size
model = QNetwork(input_size, hidden_size)